In [1]:
import sys
sys.path.append('/nobackup/samart18/edm')
sys.path.append('/nobackup/samart18/modulus')
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import xarray as xr
import numpy as np
from datetime import date, timedelta

sys.path.append('src')
from src.dataloaders import *
import math
import csv

In [2]:
data_dir = './input_data/'
ds = xr.open_dataset(data_dir + 'cmems_mod_glo_phy_my_0.083deg_P1D-m_multi-vars_70.00W-40.00W_25.00N-45.00N_0.49m_2010-01-01-2020-12-31.nc')
ds_m = xr.open_dataset(data_dir + 'glorys_gulfstream_means.nc')
ds_clim = xr.open_dataset(data_dir + 'glorys_gulfstream_climatology.nc')

var_stds = {'zos':((ds['zos']-ds_m['zos']).std()), 'thetao':((ds['thetao']-ds_clim['thetao']).std()), 'so':((ds['so']-ds_clim['so']).std()), 'uo':((ds['uo']-ds_m['uo']).std()), 'vo':((ds['vo']-ds_m['vo']).std())}

ngpu = 1
device = torch.device("cuda:0") if (torch.cuda.is_available() and ngpu > 0) else torch.device("cpu")

dataset = GAN_dataset(data_dir = './input_data/', latent_dim = 1024, n_lon = 128, n_lat = 128, samples_per_day = 10, date_range = [date(2015,1,1),date(2020,12,31)], variables = ['zos', 'thetao', 'so', 'uo', 'vo'], var_stds = var_stds, model_zarr_name = 'glorys_gulfstream_anomaly_zarr', lon_buffers = [3, None], lat_buffers = [None, 2], multiprocessing = True)
batch_size = 32
n_cpus = 2
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=n_cpus, worker_init_fn = dataset.worker_init_fn,persistent_workers=True)

/nobackupp27/samart18/GenDA/src/dataloaders.py:72: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  self.N_lon = self.ds_model.dims['longitude']
/nobackupp27/samart18/GenDA/src/dataloaders.py:73: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  self.N_lat = self.ds_model.dims['latitude']
/nobackupp27/samart18/GenDA/src/dataloaders.py:74: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  self.N_time = 

In [5]:
class InfiniteSampler(torch.utils.data.Sampler):
    def __init__(self, dataset, rank=0, num_replicas=1, shuffle=True, seed=0, window_size=0.5):
        assert len(dataset) > 0
        assert num_replicas > 0
        assert 0 <= rank < num_replicas
        assert 0 <= window_size <= 1
        super().__init__(dataset)
        self.dataset = dataset
        self.rank = rank
        self.num_replicas = num_replicas
        self.shuffle = shuffle
        self.seed = seed
        self.window_size = window_size

    def __iter__(self):
        order = np.arange(len(self.dataset))
        rnd = None
        window = 0
        if self.shuffle:
            rnd = np.random.RandomState(self.seed)
            rnd.shuffle(order)
            window = int(np.rint(order.size * self.window_size))

        idx = 0
        while True:
            i = idx % order.size
            if idx % self.num_replicas == self.rank:
                yield order[i]
            if window >= 2:
                j = (i - rnd.randint(window)) % order.size
                order[i], order[j] = order[j], order[i]
            idx += 1

In [37]:
sys.path.append('/nobackup/samart18/modulus')


In [7]:
import modulus

ImportError: cannot import name 'DEFAULT_CALLBACK' from 'fsspec.callbacks' (/nobackup/samart18/.conda/envs/GenDA/lib/python3.11/site-packages/fsspec/callbacks.py)

In [8]:
import os

In [22]:
os.listdir('/nobackup/samart18/modulus')

['.github',
 'Makefile',
 '.gitattributes',
 'Dockerfile',
 'FAQ.md',
 'README.md',
 'CONTRIBUTING.md',
 'pyproject.toml',
 'docs',
 'modulus',
 '.pre-commit-config.yaml',
 '.markdownlint.yaml',
 'CITATION.cff',
 'examples',
 '.gitignore',
 'LICENSE.txt',
 '.git',
 '.gitlab',
 '.gitmodules',
 'test',
 'CHANGELOG.md']

In [40]:
import modulus.models

ModuleNotFoundError: No module named 'modulus.models'

In [3]:
from modulus.models.diffusion.song_unet import *

ModuleNotFoundError: No module named 's3fs'

In [4]:
torch.distributed.get_world_size()

ValueError: Default process group has not been initialized, please make sure to call init_process_group.